In [ ]:
# %pip install pyarrow

In [ ]:
import pandas as pd
import numpy as np
import pickle
import os
import tensorflow_addons as tfa
import math
from math import ceil
from tensorflow.keras.utils import plot_model
from langdetect import detect

AUTO = tf.data.experimental.AUTOTUNE

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [ ]:
model_iteration = 'iteration_final'

In [ ]:
test_data = pd.read_parquet(f"/content/drive/My Drive/Colab Notebooks/mag_model/test_data_{model_iteration}/data_with_predictions.parquet")
test_data['target_test'] = test_data['target_tok'].apply(lambda x: [i for i in x if i!=-1])
test_data['target_test'] = test_data['target_test'].apply(len)
test_data = test_data[test_data['target_test'] > 0].copy()
test_data.shape

(97233, 9)

In [ ]:
test_extra_data = pd.read_parquet(f"/content/drive/My Drive/Colab Notebooks/mag_model/universal_test_data/test_extra_data.parquet") \
[['paper_id','year','month','topic_len']]
test_extra_data.shape

(150630, 4)

In [ ]:
test_raw = pd.read_parquet(f"/content/drive/My Drive/Colab Notebooks/mag_model/universal_test_data/test_raw_lang.parquet") \
[['paper_id','paper_title','topics_full','lang']]
test_raw.shape

(150630, 4)

In [ ]:
test_raw['lang'].value_counts().head(10)

en    137544
it      1926
fr      1849
es      1191
de      1107
ro       938
ru       652
ca       632
da       553
id       540
Name: lang, dtype: int64

In [ ]:
title_tags = pd.read_parquet(f"/content/drive/My Drive/Colab Notebooks/mag_model/universal_test_data/test_data_title_tagging.parquet") \
[['paper_id','title_unigrams','title_bigrams','title_trigrams','title_4grams']]
title_tags.shape

(150630, 5)

In [ ]:
test_data_all = test_data \
.merge(test_extra_data, how='left', on='paper_id') \
.merge(test_raw, how='left', on='paper_id') \
.merge(title_tags, how='left', on='paper_id')
test_data_all.shape

(97233, 19)

In [ ]:
test_data_all = test_data_all[test_data_all['lang']=='en'].copy()
test_data_all.shape

(92792, 19)

In [ ]:
test_data_all['full_topic_len'] = test_data_all['topics_full'].apply(len)
test_data_all['short_topic_len'] = test_data_all['target_tok'].apply(len)

In [ ]:
test_data_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92792 entries, 0 to 97232
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   paper_id          92792 non-null  int64 
 1   publication_date  92792 non-null  object
 2   doc_type_tok      92792 non-null  object
 3   journal_tok       92792 non-null  object
 4   target_tok        92792 non-null  object
 5   paper_title_tok   92792 non-null  object
 6   predictions       92792 non-null  object
 7   scores            92792 non-null  object
 8   target_test       92792 non-null  int64 
 9   year              92792 non-null  int32 
 10  month             92792 non-null  int32 
 11  topic_len         92792 non-null  int32 
 12  paper_title       92792 non-null  object
 13  topics_full       92792 non-null  object
 14  lang              92792 non-null  object
 15  title_unigrams    85251 non-null  object
 16  title_bigrams     58650 non-null  object
 17  title_trigra

In [ ]:
with open(f"/content/drive/My Drive/Colab Notebooks/mag_model/vocab_{model_iteration}/topics_vocab.pkl", "rb") as f:
    target_vocab = pickle.load(f)
    
target_vocab_inv = {j:i for i,j in target_vocab.items()}

with open(f"/content/drive/My Drive/Colab Notebooks/mag_model/vocab_{model_iteration}/doc_type_vocab.pkl", "rb") as f:
    doc_vocab = pickle.load(f)
    
doc_vocab_inv = {j:i for i,j in doc_vocab.items()}

with open(f"/content/drive/My Drive/Colab Notebooks/mag_model/vocab_{model_iteration}/journal_name_vocab.pkl", "rb") as f:
    journal_vocab = pickle.load(f)
    
journal_vocab_inv = {j:i for i,j in journal_vocab.items()}

with open(f"/content/drive/My Drive/Colab Notebooks/mag_model/vocab_{model_iteration}/paper_title_vocab.pkl", "rb") as f:
    title_vocab = pickle.load(f)
    
title_vocab_inv = {j:i for i,j in title_vocab.items()}


In [ ]:
print(len(target_vocab_inv))

82178


In [ ]:
test_data_all.head(1)

,paper_id,publication_date,doc_type_tok,journal_tok,target_tok,paper_title_tok,predictions,scores,target_test,year,month,topic_len,paper_title,topics_full,lang,title_unigrams,title_bigrams,title_trigrams,title_4grams
0,143700069,1976-11-01,[3],[160],"[21, 21151, 661, 25903, 65162, 161]","[23614, 16024, 32111, 68098, 50480, 628]","[661, 25903, 6793, 21151, 4291, 65162, 161, 30...","[0.5880258083343506, 0.5575599670410156, 0.433...",6,1976,11,6,nuclear safety for routine reactor operation,"[environmental science, nuclear facilities, nu...",en,[operation],None,None,None


In [ ]:
test_data_all['topics_target'] = test_data_all['target_tok'].apply(lambda x: [target_vocab_inv.get(i) for i in x])
test_data_all['predictions_text'] = test_data_all['predictions'].apply(lambda x: [target_vocab_inv.get(i) for i in x])

In [ ]:
encoding_layer = tf.keras.layers.experimental.preprocessing.CategoryEncoding(
    max_tokens=len(target_vocab)+1, output_mode="binary", sparse=False)

In [ ]:
def thresh_preds(preds, scores, threshold):
  new_preds = [x for x,y in zip(preds, scores) if y >= threshold]
  if len(new_preds) == 0:
    new_preds = preds[:1]
  return new_preds

In [ ]:
# for thresh in [5,10,15,20,25,30,35,40,45,50,55,60]:
for thresh in [30,32,35]:
  print(f"--------------Threshold: {thresh}--------------")
  test_data_all[f'predictions_{str(thresh)}'] = test_data_all.apply(lambda x: thresh_preds(x.predictions, x.scores, thresh/100), axis=1)
  test_data_all[f'predictions_{str(thresh)}_text'] = test_data_all[f'predictions_{str(thresh)}'].apply(lambda x: [target_vocab_inv.get(i) for i in x])

--------------Threshold: 30--------------
--------------Threshold: 32--------------
--------------Threshold: 35--------------


## Set the threshold here and then run code below

Need to run prediction code first before running the cell with "for_examples"

In [ ]:
threshold=32

In [ ]:
for_examples = test_data_all[['paper_id','target_tok','predictions',f'predictions_{thresh}']].merge(test_raw[['paper_id','paper_title']], 
                                                                                                how='left', on='paper_id').sample(500).reset_index(drop=True)

In [ ]:
for i in range(500):
  print(for_examples.iloc[i,0])
  print(for_examples.iloc[i,-1])
  print("\n")
  print(f"Tags: {[target_vocab_inv.get(x) for x in for_examples.iloc[i,1]]}")
  print("\n")
  print(f"Predictions: {[target_vocab_inv.get(x) for x in for_examples.iloc[i,3]]}")
  print("----------------------------------------------------------------------")
  print("\n")

Streaming output truncated to the last 5000 lines.


Predictions: ['function', 'computer science']
----------------------------------------------------------------------


1986366352
paraganglioma of the urinary bladder a lesion that may be misdiagnosed as urothelial carcinoma in transurethral resection specimens


Tags: ['urinary bladder', 'medicine', 'urinary system', 'differential diagnosis', 'transitional cell carcinoma', 'pathology', 'paraganglioma', 'cystoscopy', 'carcinoma']


Predictions: ['medicine', 'urinary bladder', 'paraganglioma', 'urology', 'radiology', 'resection', 'carcinoma', 'cystoscopy', 'pathology', 'bladder neoplasm']
----------------------------------------------------------------------


2081063591
neogene extensional deformation and related stress regimes in central tunisia


Tags: ['neogene', 'paleontology', 'subduction', 'context', 'geology', 'block', 'rift', 'seismology', 'quaternary', 'late miocene', 'submarine pipeline']


Predictions: ['geology', 'extensi

### Getting Metrics

In [ ]:
def get_metrics_text(data, target_col, predict_col):
  targets = data[target_col].to_list()
  predictions = data[predict_col].to_list()

  num_tags = len(target_vocab)
  num_topics = 718000
  num_targets = len([x for y in targets for x in y])
  TP = 0
  TN = 0
  FP = 0
  FN = 0
  weighted_acc = 0
  for i,j in zip(predictions, targets):
    try:
      tp_temp = len([x for x in i if x in j])
      fp_temp = len([x for x in i if x not in j])
      fn_temp = len([x for x in j if x not in i])
      weighted_acc += tp_temp/len(j)
      TP += tp_temp
      FP += fp_temp
      FN += fn_temp
      TN += (num_tags - TP - FP - FN)
    except:
      print(i)
      print(j)
      print(type(i))
      print(type(j))
  
  recall = TP/(TP+FN)
  precision = TP/(TP+FP)
  f1_score = 2*(recall*precision)/(recall+precision)
  accuracy = TP/num_targets
  weighted_acc = weighted_acc/len(predictions)

  print(f"Recall: {round(recall*100, 1)}%")
  print(f"Precision: {round(precision*100, 1)}%")
  print(f"F1 Score: {round(f1_score*100, 1)}%")
  print(f"Accuracy: {round(accuracy*100, 1)}%")
  print(f"Weighted Accuracy: {round(weighted_acc*100, 1)}%")

  return TP,FP,TN,FN


### Looking at different probability thresholds

In [ ]:
for thresh in [30,32,35]:
  print(f"--------------Threshold: {thresh}--------------")
  _ = get_metrics_text(test_data_all, 'topics_target', f'predictions_{thresh}_text')
  print("\n")

--------------Threshold: 30--------------
Recall: 49.6%
Precision: 37.2%
F1 Score: 42.5%
Accuracy: 49.6%
Weighted Accuracy: 52.4%


--------------Threshold: 32--------------
Recall: 46.2%
Precision: 42.5%
F1 Score: 44.3%
Accuracy: 46.2%
Weighted Accuracy: 48.9%


--------------Threshold: 35--------------
Recall: 41.1%
Precision: 50.8%
F1 Score: 45.4%
Accuracy: 41.1%
Weighted Accuracy: 43.7%




### Levels

In [ ]:
levels_df = pd.read_parquet(f"/content/drive/My Drive/Colab Notebooks/mag_model/universal_test_data/tag_levels.parquet").fillna(6)
levels_df['level'] = levels_df['level'].astype('int')

In [ ]:
def fill_blank(arr):
  if not type(arr) == list:
    arr=["NO TAGS FOR THIS"]
  return arr

In [ ]:
def get_df_for_specific_level(old_df, levels, level_to_get=1, pred_col="predictions", target_col='topics_target'):
    df = old_df.copy()
    curr_topics = pd.DataFrame(zip(target_vocab.keys()), columns=['topic_name'])
    tags_list = levels[levels['level']==level_to_get].merge(curr_topics, how='inner', on='topic_name')['topic_name'].to_list()
    # tags_id_list = [target_vocab.get(x) for x in tags_list]
    print(f"Number of Tags: {len(tags_list)}")
    
    # df[f'tags_level_{level_to_get}'] = df['target_tok'].apply(lambda x: [i for i in x if i in tags_id_list])
    # df[f'preds_level_{level_to_get}'] = df[pred_col].apply(lambda x: [i for i in x if i in tags_id_list])

    df_preds = df.explode(pred_col).merge(levels[levels['level']==level_to_get], 
                                          how='inner', left_on=pred_col, right_on='topic_name')
    
    df_targs = df.explode(target_col).merge(levels[levels['level']==level_to_get], 
                                            how='inner', left_on=target_col, right_on='topic_name')
    
    df_preds_1 = df_preds[['paper_id',pred_col]].groupby('paper_id')[pred_col].apply(list).reset_index()
    df_targs_2 = df_targs[['paper_id',target_col]].groupby('paper_id')[target_col].apply(list).reset_index()

    new_df = df_preds_1.merge(df_targs_2, how='outer', on='paper_id')
    
    papers_with_tag = (new_df[~new_df[target_col].isnull()].shape[0])/df.shape[0]
    papers_with_pred = (new_df[~new_df[pred_col].isnull()].shape[0])/df.shape[0]

    new_df[target_col] = new_df[target_col].apply(lambda x: fill_blank(x))
    new_df[pred_col] = new_df[pred_col].apply(lambda x: fill_blank(x))
    
    print(f"Percentage of papers with Level {level_to_get} Tags: {round(papers_with_tag*100, 1)}")
    print(f"Percentage of papers with Level {level_to_get} Preds: {round(papers_with_pred*100, 1)}")
    
    return new_df

In [ ]:
thresh = 32
print(f"Threshold: {thresh}")
for level_to_check in range(0,6):
  print(f"Topic Level: {level_to_check}")
  df = get_df_for_specific_level(test_data_all, levels_df, level_to_check, f"predictions_{thresh}_text")
  get_metrics_text(df, 'topics_target', f'predictions_{thresh}_text')
  print("------------------------------------------------------------")
  print("\n")

Threshold: 32
Topic Level: 0
Number of Tags: 19
Percentage of papers with Level 0 Tags: 99.4
Percentage of papers with Level 0 Preds: 99.3
Recall: 87.4%
Precision: 64.5%
F1 Score: 74.2%
Accuracy: 87.4%
Weighted Accuracy: 87.4%
------------------------------------------------------------


Topic Level: 1
Number of Tags: 291
Percentage of papers with Level 1 Tags: 99.6
Percentage of papers with Level 1 Preds: 92.3
Recall: 63.1%
Precision: 42.9%
F1 Score: 51.0%
Accuracy: 63.0%
Weighted Accuracy: 64.7%
------------------------------------------------------------


Topic Level: 2
Number of Tags: 33176
Percentage of papers with Level 2 Tags: 95.6
Percentage of papers with Level 2 Preds: 95.3
Recall: 34.1%
Precision: 39.6%
F1 Score: 36.6%
Accuracy: 34.1%
Weighted Accuracy: 39.0%
------------------------------------------------------------


Topic Level: 3
Number of Tags: 31029
Percentage of papers with Level 3 Tags: 80.6
Percentage of papers with Level 3 Preds: 75.1
Recall: 32.9%
Precision: 3

### Journal and Doc Types

In [ ]:
test_data_all['journal'] = test_data_all['journal_tok'].apply(lambda x: [journal_vocab_inv.get(i) for i in x][0])
test_data_all['doc_type'] = test_data_all['doc_type_tok'].apply(lambda x: [doc_vocab_inv.get(i) for i in x][0])

In [ ]:
thresh=32
for doc_type in test_data_all['doc_type'].value_counts().index:
    print(doc_type)
    # num_samples = test_data_all[test_data_all['doc_type']==doc_type].shape[0]
    # if num_samples >= 5000:
    #   num_to_sample = 5000
    # else:
    #   num_to_sample = num_samples
    get_metrics_text(test_data_all[test_data_all['doc_type']==doc_type], "topics_target", f"predictions_{thresh}_text")
    print("-----------------------------------------------------------------------")
    print("\n")

Journal
Recall: 46.4%
Precision: 42.4%
F1 Score: 44.3%
Accuracy: 46.3%
Weighted Accuracy: 48.7%
-----------------------------------------------------------------------


Conference
Recall: 41.8%
Precision: 42.5%
F1 Score: 42.1%
Accuracy: 41.7%
Weighted Accuracy: 45.1%
-----------------------------------------------------------------------


Repository
Recall: 41.2%
Precision: 43.5%
F1 Score: 42.3%
Accuracy: 41.2%
Weighted Accuracy: 44.6%
-----------------------------------------------------------------------


Book
Recall: 45.5%
Precision: 46.7%
F1 Score: 46.1%
Accuracy: 45.5%
Weighted Accuracy: 53.9%
-----------------------------------------------------------------------


BookChapter
Recall: 55.1%
Precision: 47.5%
F1 Score: 51.1%
Accuracy: 55.1%
Weighted Accuracy: 62.3%
-----------------------------------------------------------------------


Thesis
Recall: 49.1%
Precision: 45.1%
F1 Score: 47.0%
Accuracy: 49.1%
Weighted Accuracy: 54.1%
------------------------------------------------

### Paper Title Length

In [ ]:
# 0 - 20, 20 - 40

In [ ]:
test_data_all['paper_title_tok_len'] = test_data_all['paper_title_tok'].apply(len)

In [ ]:
(test_data_all[test_data_all['paper_title_tok_len'] < 10].shape[0])/test_data_all.shape[0]

0.2170876799724114

In [ ]:
(test_data_all[(test_data_all['paper_title_tok_len'] >= 10) & (test_data_all['paper_title_tok_len']<20)].shape[0])/test_data_all.shape[0]

0.6593133028709371

In [ ]:
(test_data_all[(test_data_all['paper_title_tok_len'] >= 20) & (test_data_all['paper_title_tok_len']<40)].shape[0])/test_data_all.shape[0]

0.12247823088197259

In [ ]:
get_metrics_text(test_data_all[test_data_all['paper_title_tok_len'] < 10], "topics_target", f"predictions_{thresh}_text")

Recall: 42.6%
Precision: 44.0%
F1 Score: 43.3%
Accuracy: 42.6%
Weighted Accuracy: 48.2%


(61927, 78684, -583565450, 83455)

In [ ]:
get_metrics_text(test_data_all[(test_data_all['paper_title_tok_len'] >= 10) & (test_data_all['paper_title_tok_len']<20)], "topics_target", f"predictions_{thresh}_text")

Recall: 46.3%
Precision: 42.2%
F1 Score: 44.1%
Accuracy: 46.3%
Weighted Accuracy: 48.5%


(244651, 335596, -21281900351, 283925)

In [ ]:
get_metrics_text(test_data_all[(test_data_all['paper_title_tok_len'] >= 20) & (test_data_all['paper_title_tok_len']<40)], "topics_target", f"predictions_{thresh}_text")

Recall: 50.7%
Precision: 42.3%
F1 Score: 46.1%
Accuracy: 50.7%
Weighted Accuracy: 52.0%


(51041, 69585, -30729871, 49572)

### Time

In [ ]:
thresh=32

In [ ]:
explore_time = test_data_all.copy()

In [ ]:
explore_time['yearMonth'] = explore_time['year']*100+explore_time['month']

In [ ]:
get_metrics_text(explore_time[explore_time['year'] <= 2002], "topics_target", f"predictions_{thresh}_text")

Recall: 48.6%
Precision: 45.7%
F1 Score: 47.1%
Accuracy: 48.6%
Weighted Accuracy: 53.3%


(57965, 68772, -192191004, 61267)

In [ ]:
get_metrics_text(explore_time[(explore_time['year'] >= 2003) & (explore_time['year'] <= 2008)], "topics_target", f"predictions_{thresh}_text")

Recall: 45.3%
Precision: 44.7%
F1 Score: 45.0%
Accuracy: 45.3%
Weighted Accuracy: 48.2%


(27390, 33826, 256932715, 33091)

In [ ]:
get_metrics_text(explore_time[(explore_time['year'] >= 2009) & (explore_time['year'] <= 2014)], "topics_target", f"predictions_{thresh}_text")

Recall: 44.7%
Precision: 44.5%
F1 Score: 44.6%
Accuracy: 44.7%
Weighted Accuracy: 47.3%


(40127, 49970, 128401399, 49592)

In [ ]:
get_metrics_text(explore_time[(explore_time['year'] >= 2015) & (explore_time['year'] <= 2020)], "topics_target", f"predictions_{thresh}_text")

Recall: 46.2%
Precision: 43.1%
F1 Score: 44.6%
Accuracy: 46.2%
Weighted Accuracy: 49.5%


(41805, 55120, 105722713, 48708)

In [ ]:
get_metrics_text(explore_time[(explore_time['year'] == 2021) & (explore_time['yearMonth'] < 202106)], "topics_target", f"predictions_{thresh}_text")

Recall: 47.5%
Precision: 41.5%
F1 Score: 44.3%
Accuracy: 47.5%
Weighted Accuracy: 50.9%


(2926, 4126, 60837144, 3235)

In [ ]:
get_metrics_text(explore_time[(explore_time['year'] == 2021) & (explore_time['yearMonth'] >= 202106)], "topics_target", f"predictions_{thresh}_text")

Recall: 45.9%
Precision: 40.8%
F1 Score: 43.2%
Accuracy: 45.9%
Weighted Accuracy: 47.6%


(187749, 272389, -12046043757, 221370)

### Looking at L0 Topic Performance

In [ ]:
def get_df_for_all_levels(old_df, levels, topic_to_get, pred_col="predictions", target_col='topics_target'):
    df = old_df.copy()
    # curr_topics = pd.DataFrame(zip(target_vocab.keys()), columns=['topic_name'])
    # tags_list = levels[levels['level']==0].merge(curr_topics, how='inner', on='topic_name')['topic_name'].to_list()
    # print(f"Number of Tags: {len(tags_list)}")

    df_preds = df.explode(pred_col)
    papers_with_topic_preds = df_preds[df_preds[pred_col]==topic_to_get][['paper_id']]
    df_preds = df.merge(papers_with_topic_preds, how='inner', on='paper_id')
    
    df_targs = df.explode(target_col)
    papers_with_topic_targs = df_targs[df_targs[target_col]==topic_to_get][['paper_id']]
    df_targs = df.merge(papers_with_topic_targs, how='inner', on='paper_id')
    
    df_preds_1 = df_preds[['paper_id',pred_col]] #.groupby('paper_id')[pred_col].apply(list).reset_index()
    df_targs_2 = df_targs[['paper_id',target_col]] #.groupby('paper_id')[target_col].apply(list).reset_index()

    new_df = df_preds_1.merge(df_targs_2, how='outer', on='paper_id')
    
    papers_with_tag = (new_df[~new_df[target_col].isnull()].shape[0])/df.shape[0]
    papers_with_pred = (new_df[~new_df[pred_col].isnull()].shape[0])/df.shape[0]

    final_df = df.merge(pd.concat([df_preds, df_targs], axis=0).drop_duplicates(subset=['paper_id'])[['paper_id']], 
                        how='inner', on='paper_id')

    final_df[target_col] = final_df[target_col].apply(lambda x: fill_blank(x))
    final_df[pred_col] = final_df[pred_col].apply(lambda x: fill_blank(x))
    
    print(f"Percentage of papers with Topic {topic_to_get} Tags: {round(papers_with_tag*100, 1)}")
    print(f"Percentage of papers with Topic {topic_to_get} Preds: {round(papers_with_pred*100, 1)}")
    
    return final_df

In [ ]:
thresh=32
for topic in levels_df[levels_df['level']==0]['topic_name'].to_list():
  print(f"Topic: {topic}")
  df = get_df_for_all_levels(test_data_all, levels_df, topic, f'predictions_{thresh}_text')
  get_metrics_text(df, 'topics_target', f'predictions_{thresh}_text')
  print("------------------------------------------------------------")
  print("\n")

Topic: economics
Percentage of papers with Topic economics Tags: 1.8
Percentage of papers with Topic economics Preds: 2.8
Recall: 36.3%
Precision: 38.2%
F1 Score: 37.2%
Accuracy: 36.3%
Weighted Accuracy: 39.1%
------------------------------------------------------------


Topic: physics
Percentage of papers with Topic physics Tags: 4.1
Percentage of papers with Topic physics Preds: 5.5
Recall: 43.0%
Precision: 40.1%
F1 Score: 41.5%
Accuracy: 43.0%
Weighted Accuracy: 45.3%
------------------------------------------------------------


Topic: materials science
Percentage of papers with Topic materials science Tags: 17.3
Percentage of papers with Topic materials science Preds: 22.5
Recall: 48.6%
Precision: 42.1%
F1 Score: 45.1%
Accuracy: 48.6%
Weighted Accuracy: 49.9%
------------------------------------------------------------


Topic: medicine
Percentage of papers with Topic medicine Tags: 14.5
Percentage of papers with Topic medicine Preds: 17.5
Recall: 49.5%
Precision: 44.1%
F1 Score: